# Turn Tiff Image series into Tiff Image stack file

This code was developed to turn frames from miniscope recordings into virtual stack TIFF files.

It opens batches of 1000 frames at a time, ordered by the data modification time (<b> Do not change individual frames frames, otherwise code will not work! </b>).

The code opens 1000 frames each time and saves it into a multiseries big tiff file. This is done to not overload the RAM memory of small RAM computers.
The name of the file will be the name of the folder in which the frames are stored.

## Load packages

This code relies on tifffile, os and numpy packages. The package time is used to show the time that the conversion is taking, so that users have an idea of the amount of time it will still take.

In [1]:
import logging
try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.INFO)

import tifffile
import os
import numpy as np
import time

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_2128\618434360.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_2128\618434360.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')


### Open series of images and sort them by recording time

The series of frame have to be saved in one folder only. Put the name of the directory in the dirpath variable (<b> Have into account to use '/' or '\\' instead of the normally pasted '/' between folders </b>).

The file names are then sorted by last modified date to be stacked in the next step

In [13]:
dirpath = 'Z:/Calcium_Analysis/12937-06_S/12937-06AI_7DPI_S'

t0 = time.time()

files = os.listdir(dirpath)
files = [os.path.join(dirpath, f) for f in files]
files.sort(key=os.path.getctime)

logging.info("Sorted " + str(len(files)) + " in " + str(int(time.time() - t0)) + " seconds")

    18583518 [3141525202.py:            <module>():9] [2128] Sorted 45000 in 348 seconds


### Open every frame. 

As said above, open stacks of 1000 frames and save them into a TIFF file with the name of the folder of all the frames

In [14]:
saveFilename = os.path.join(os.path.dirname(dirpath),os.path.basename(dirpath)+'.tif')           # Name of image stack to save
n=0                                # Counter of number of frames before saving (until 1000)
nbits = np.uint8                   # Precision in which to save frames. Ninscope/Miniscope data is 8 bit.

logging.info("Starting to convert frames from: "+ dirpath + " into " + saveFilename)
t0 = time.time()
for i in range(0, len(files)):
    
    frame = tifffile.TiffFile(files[i]).asarray()        # Open frame
    if i == 0:
        images = np.zeros((1000,frame.shape[0], frame.shape[1]))     # Initialize variable to store opened frames
    images[n] = frame
    
    if n == 999:
        if i== 999:
            append = False
        else:
            append = True
            
        tifffile.imwrite(saveFilename, images.astype(nbits), append = append, bigtiff = True)
        logging.info ("Saved " + str(i+1) + " of " + str(len(files)) + " frames in " + str(int(time.time()-t0)) + " seconds.")
        n=0
    else:
        n +=1
        
logging.info ("Saved all frames in " + str(int(time.time()-t0)) + " seconds.")
    

    18583753 [1696955182.py:            <module>():5] [2128] Starting to convert frames from: Z:/Calcium_Analysis/12937-06_S/12937-06AI_7DPI_S into Z:/Calcium_Analysis/12937-06_S\12937-06AI_7DPI_S.tif
    18648824 [1696955182.py:            <module>():21] [2128] Saved 1000 of 45000 frames in 65 seconds.
    18684020 [1696955182.py:            <module>():21] [2128] Saved 2000 of 45000 frames in 100 seconds.
    18711421 [1696955182.py:            <module>():21] [2128] Saved 3000 of 45000 frames in 127 seconds.
    18738727 [1696955182.py:            <module>():21] [2128] Saved 4000 of 45000 frames in 154 seconds.
    18763760 [1696955182.py:            <module>():21] [2128] Saved 5000 of 45000 frames in 180 seconds.
    18795132 [1696955182.py:            <module>():21] [2128] Saved 6000 of 45000 frames in 211 seconds.
    18839273 [1696955182.py:            <module>():21] [2128] Saved 7000 of 45000 frames in 255 seconds.
    18883555 [1696955182.py:            <module>():21] [2128] Sav